In [660]:
import pandas as pd
import numpy as np
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression

In [661]:
# load dataframes
games = pd.read_csv("data/games.csv")
plays = pd.read_csv("data/plays.csv")
tackles = pd.read_csv("data/tackles.csv")
players = pd.read_csv("data/players.csv")


In [662]:
# add target var (can also use play desc to count scrambles as runs)
plays["run"] = plays["passResult"].isna()

In [663]:
#makes pass result binary(1 run, 0 pass)
plays["run"] = plays["passResult"].isna()
#filters play data
plays_filtered = plays[["gameId","quarter", "down", "yardsToGo", "possessionTeam","defensiveTeam","absoluteYardlineNumber", "gameClock", "preSnapHomeScore",
                        "preSnapVisitorScore", "run","offenseFormation", "defendersInTheBox","expectedPoints"]]

In [664]:
#filters game data
games_filtered = games[["gameId", "week","gameTimeEastern"]]

In [665]:
# filter players dataframe
players_filtered = players[["position", "nflId"]]

In [666]:
# filter tackles
tackles_filtered = tackles[["gameId", "playId", "tackle", "assist", "forcedFumble", "pff_missedTackle"]]
tackles_filtered = tackles_filtered.groupby(["gameId", "playId"]).sum().reset_index()

In [667]:
#combine game data with play data
data = plays_filtered.merge(games_filtered, on="gameId")

In [668]:
#make all game times purly numbers (probably not necessary - most models can handle datetime or can convert to int differently)
data['gameClock'] = data['gameClock'].replace(':', '', regex =True).astype(int)
data['gameTimeEastern'] = data['gameTimeEastern'].replace(':', '', regex =True).astype(int)
data.drop("gameId",axis =1, inplace=True)
#normalization]
for param in ['yardsToGo','absoluteYardlineNumber',	'gameClock',	'preSnapHomeScore',	'preSnapVisitorScore',	'defendersInTheBox','gameTimeEastern']:
    data[param] = (data[param] - data[param].mean())  +.5
    data[param] = (data[param] - data[param].mean())  +.5

# def str_to_time(time):
#     splt = time.split(":")
#     return int(splt[0]) * 60 + int(splt[1])
# data["gameTimeEastern"] = data["gameTimeEastern"].apply(str_to_time)
# data["gameClock"] = data["gameClock"].apply(str_to_time)

In [669]:
#find all unique teams
teams = data.possessionTeam.unique()
#1 hot encode teams and formations
data = pd.get_dummies(data, prefix=['possessionTeam', 'defensiveTeam', 'offenseFormation'])
data.fillna(0,inplace=True)

In [670]:
#empty dictionary of team names
data_by_team_test = {x : pd.DataFrame() for x in teams}
data_by_team_train = {x : pd.DataFrame() for x in teams}
data_test = pd.DataFrame()
data_train = pd.DataFrame()
#fills dictionary with all plays according to possesion team
for team in teams:
    data_by_team_test[team] = data[data["possessionTeam_"+str(team)]]
    #seperate into train and test sets seperated by team 
    rows = len(data_by_team_test[team])
    rand_idx = np.random.randint(0, rows,size = int(rows/10))
    #make seperate train and test sets for each team
    data_by_team_train[team] = data_by_team_test[team].drop(data_by_team_test[team].index[rand_idx])
    data_by_team_test[team] = data_by_team_test[team].iloc[rand_idx]
    #create a joined train and test set(for overall)
    data_test = pd.concat([data_test, data_by_team_test[team]], ignore_index = True)
    data_train = pd.concat([data_train, data_by_team_train[team]], ignore_index = True)

#Randomize final sets to mix teams
data_test = data_test.sample(frac=1)
data_train = data_train.sample(frac=1)
#split datasets into parameters and result
data_test_run = data_test["run"]
data_test_epa = data_test["expectedPoints"]
data_test.drop(["run","expectedPoints"],axis =1, inplace=True)
data_train_run = data_train["run"]
data_train_epa = data_train["expectedPoints"]
data_train.drop(["run","expectedPoints"],axis =1, inplace=True)
data_test

,quarter,down,yardsToGo,absoluteYardlineNumber,gameClock,preSnapHomeScore,preSnapVisitorScore,defendersInTheBox,week,gameTimeEastern,possessionTeam_ARI,possessionTeam_ATL,possessionTeam_BAL,possessionTeam_BUF,possessionTeam_CAR,possessionTeam_CHI,possessionTeam_CIN,possessionTeam_CLE,possessionTeam_DAL,possessionTeam_DEN,possessionTeam_DET,possessionTeam_GB,possessionTeam_HOU,possessionTeam_IND,possessionTeam_JAX,possessionTeam_KC,possessionTeam_LA,possessionTeam_LAC,possessionTeam_LV,possessionTeam_MIA,possessionTeam_MIN,possessionTeam_NE,possessionTeam_NO,possessionTeam_NYG,possessionTeam_NYJ,possessionTeam_PHI,possessionTeam_PIT,possessionTeam_SEA,possessionTeam_SF,possessionTeam_TB,possessionTeam_TEN,possessionTeam_WAS,defensiveTeam_ARI,defensiveTeam_ATL,defensiveTeam_BAL,defensiveTeam_BUF,defensiveTeam_CAR,defensiveTeam_CHI,defensiveTeam_CIN,defensiveTeam_CLE,defensiveTeam_DAL,defensiveTeam_DEN,defensiveTeam_DET,defensiveTeam_GB,defensiveTeam_HOU,defensiveTeam_IND,defensiveTeam_JAX,defensiveTeam_KC,defensiveTeam_LA,defensiveTeam_LAC,defensiveTeam_LV,defensiveTeam_MIA,defensiveTeam_MIN,defensiveTeam_NE,defensiveTeam_NO,defensiveTeam_NYG,defensiveTeam_NYJ,defensiveTeam_PHI,defensiveTeam_PIT,defensiveTeam_SEA,defensiveTeam_SF,defensiveTeam_TB,defensiveTeam_TEN,defensiveTeam_WAS,offenseFormation_EMPTY,offenseFormation_I_FORM,offenseFormation_JUMBO,offenseFormation_PISTOL,offenseFormation_SHOTGUN,offenseFormation_SINGLEBACK,offenseFormation_WILDCAT
1130,3,2,1.030915,12.073923,-400.586016,14.329329,25.637033,1.075915,9,-21371.596748,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False
214,1,1,2.030915,-24.926077,321.413984,-3.670671,-9.362967,0.075915,9,-21371.596748,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False
390,3,1,2.030915,-4.926077,-362.586016,6.329329,14.637033,0.075915,4,-21371.596748,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False
1194,4,1,2.030915,-26.926077,-211.586016,-1.670671,10.637033,0.075915,3,11128.403252,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False
198,4,2,2.030915,-14.926077,97.413984,5.329329,20.637033,-1.924085,7,11128.403252,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False

In [671]:
data_test_run = np.ravel(np.array(data_test_run))
data_test_epa = np.ravel(np.array(data_test_epa))
data_test_param = np.array(data_test)
data_train_run = np.ravel(np.array(data_train_run))
data_train_epa = np.ravel(np.array(data_train_epa))
data_train_param = np.array(data_train)

In [672]:
# initial random forest classifier
clf = RandomForestClassifier()

In [673]:
clf.fit(data_train, data_train_run)

RandomForestClassifier()

In [674]:
preds = clf.predict(data_test)

In [675]:
acc = (preds == data_test_run).sum() / len(preds)
acc * 100

74.0470397404704

In [676]:
model = LinearRegression()
model.fit(data_train_param, data_train_run)
pass_pred = model.predict(data_test_param)
pass_pred = np.where(pass_pred>.5,1,0)
matching = np.sum(pass_pred == data_test_run)
#percentage accuarcy
matching/len(pass_pred) * 100

72.83049472830496